https://github.com/mohamedelhakim/Transformer-CNN-Architecture

# Import modules

In [65]:
import pandas as pd
import numpy as np
import os
import torch
from torch import nn, optim
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset

In [66]:
import matplotlib.pyplot as plt

In [67]:
# X=rnnamp_model['text']
# y=np.array(rnnamp_model['labels'])




In [68]:
# rnnamp_model = pd.read_csv('/kaggle/input/hemolytic/combined.csv')

dataframes = {}
directory_path = 'Heartbeat_Dataset'
all_files = os.listdir(directory_path)

for file in os.listdir(directory_path):
    if file.endswith('.csv'):
        file_path = os.path.join(directory_path, file)
        # Remove the .csv extension for the DataFrame name
        df_name = os.path.splitext(file)[0]
        dataframes[df_name] = pd.read_csv(file_path, header= None)



In [69]:
train_df = dataframes['mitbih_train']

In [70]:
labels = {
    0.0: "N",
    1.0: "S",
    2.0: "V",
    3.0: "F",
    4.0: "Q"
}

train_df.iloc[:, -1] = train_df.iloc[:, -1].replace(labels)

# Now get the value counts for the renamed last column
train_counts = train_df.iloc[:, -1].value_counts()

# Print the counts
print(train_counts)

187
N    72471
Q     6431
V     5788
S     2223
F      641
Name: count, dtype: int64


C:\Users\randa\AppData\Local\Temp\ipykernel_10380\4019137323.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0        N
1        N
2        N
3        N
4        N
        ..
87549    Q
87550    Q
87551    Q
87552    Q
87553    Q
Name: 187, Length: 87554, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  train_df.iloc[:, -1] = train_df.iloc[:, -1].replace(labels)


In [71]:
train_df

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,0.977941,0.926471,0.681373,0.245098,0.154412,0.191176,0.151961,0.085784,0.058824,0.049020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N
1,0.960114,0.863248,0.461538,0.196581,0.094017,0.125356,0.099715,0.088319,0.074074,0.082621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N
2,1.000000,0.659459,0.186486,0.070270,0.070270,0.059459,0.056757,0.043243,0.054054,0.045946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N
3,0.925414,0.665746,0.541436,0.276243,0.196133,0.077348,0.071823,0.060773,0.066298,0.058011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N
4,0.967136,1.000000,0.830986,0.586854,0.356808,0.248826,0.145540,0.089202,0.117371,0.150235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87549,0.807018,0.494737,0.536842,0.529825,0.491228,0.484211,0.456140,0.396491,0.284211,0.136842,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q
87550,0.718333,0.605000,0.486667,0.361667,0.231667,0.120000,0.051667,0.001667,0.000000,0.013333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q
87551,0.906122,0.624490,0.595918,0.575510,0.530612,0.481633,0.444898,0.387755,0.322449,0.191837,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q
87552,0.858228,0.645570,0.845570,0.248101,0.167089,0.131646,0.121519,0.121519,0.118987,0.103797,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Q


In [72]:
x_data = train_df.iloc[:,:187]
y_label = train_df.iloc[:,-1]

In [73]:
y_label.value_counts()

187
N    72471
Q     6431
V     5788
S     2223
F      641
Name: count, dtype: int64

In [74]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_label)  # Encode labels without replacement
num_classes = len(label_encoder.classes_)  # Use the number of unique classes
y = np.eye(num_classes)[y] 

In [75]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y, test_size=0.2, random_state=42)

In [76]:
X_train = np.expand_dims(X_train, axis=1)  # Change axis from 2 to 1
X_test = np.expand_dims(X_test, axis=1)    # Change axis from 2 to 1

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train).float()  # Ensure the data is of float type
X_test_tensor = torch.tensor(X_test).float()    # Ensure the data is of float type
y_train_tensor = torch.tensor(y_train).long()   # Ensure labels are long type
y_test_tensor = torch.tensor(y_test).long()   

y_train_tensor = y_train_tensor.argmax(dim=1)  # Reduces [70043, 5] to [70043]
y_test_tensor = y_test_tensor.argmax(dim=1)
# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [77]:
print("X_train_tensor shape:", X_train_tensor.shape)  #
print("y_train_tensor shape:", y_train_tensor.shape) 

X_train_tensor shape: torch.Size([70043, 1, 187])
y_train_tensor shape: torch.Size([70043])


In [78]:
print(train_dataset)
print(train_loader)

In [79]:
class CNNFeatureExtractor(nn.Module):
    def __init__(self, num_classes):
        super(CNNFeatureExtractor, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3)
        
        # Max pooling
        self.pool = nn.MaxPool1d(kernel_size=2)
        
        # Flatten layer
        self.flatten = nn.Flatten()
        
        # Fully connected layers
        self.fc1 = nn.Linear(256 * self._get_conv_output_size(187), 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)

    def _get_conv_output_size(self, input_length):
        output_length = input_length
        # Calculate the output length after each convolution and pooling operation
        for _ in range(3):  # Three convolution layers
            output_length = (output_length - 2) // 2  # Pooling reduces the length by half after each convolution
        return output_length

    def forward(self, x):
        # Apply convolutional layers followed by ReLU and pooling
        x = self.pool(F.relu(self.conv1(x)))  # First convolution + pooling
        x = self.pool(F.relu(self.conv2(x)))  # Second convolution + pooling
        x = self.pool(F.relu(self.conv3(x)))  # Third convolution + pooling
        
        # Flatten the tensor output before passing it through the fully connected layers
        x = self.flatten(x)
        
        # Fully connected layer + dropout
        x = F.relu(self.fc1(x))  # First fully connected layer
        x = self.dropout(x)      # Dropout layer
        x = self.fc2(x)          # Final output layer
        
        return x





        
# CNN-Transformer Hybrid Model
class CNNTransformerHybrid(nn.Module):
    def __init__(self, input_dim, num_classes, num_heads, num_layers):
        super(CNNTransformerHybrid, self).__init__()

        # CNN Feature extractor with Conv1d
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(2)
        
        # Transformer Encoder setup
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=128, nhead=num_heads), num_layers=num_layers
        )
        
        # Fully connected layer to output the final classification
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        if x.ndim == 2:
            x = x.unsqueeze(1)  # Add channel dimension: (batch_size, 1, seq_len)
        x = self.pool(F.relu(self.conv1(x)))  # First Conv1d layer + Pooling
        x = self.pool(F.relu(self.conv2(x)))  # Second Conv1d layer + Pooling
        
        # Step 2: Transformer input preparation
        x = x.permute(2, 0, 1)  # Change shape to (seq_len, batch_size, features) for the transformer
        x = self.encoder(x)  # Transformer Encoder
        
        # Step 3: Classification
        x = x.mean(dim=0)  # Aggregate over sequence length
        x = self.fc(x)  # Fully connected layer to get final class prediction
        
        return x

# Instantiation
model = CNNTransformerHybrid(
    input_dim=187,  # Number of features per sample (input dimension)
    num_classes=10,  # Number of output classes
    num_heads=8,  # Multi-head attention heads
    num_layers=6  # Number of Transformer layers
)


c:\Users\randa\OneDrive\Documents\GitHub\Neural-Networks-Project\.venv\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [80]:
# Print model summary
print(model)

CNNTransformerHybrid(
  (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc): Linear(in_features=128, out_features

In [81]:
# Define a simple training and testing loop
def train(model, train_loader, optimizer, criterion, device):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        output = model(data)
        
        # Calculate the loss
        loss = criterion(output, target)
        
        # Backward pass
        loss.backward()
        
        # Optimize the weights
        optimizer.step()
        
        # Update the running loss
        running_loss += loss.item()
        
        # Get the predicted class
        _, predicted = torch.max(output, 1)
        
        # Update the total and correct count
        total += target.size(0)
        correct += (predicted == target).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


def test(model, test_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():  # No need to compute gradients during testing
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            # Forward pass
            output = model(data)
            
            # Calculate the loss
            loss = criterion(output, target)
            running_loss += loss.item()
            
            # Get the predicted class
            _, predicted = torch.max(output, 1)
            
            total += target.size(0)
            correct += (predicted == target).sum().item()
            
            # Save predictions and labels for additional evaluation if needed
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(target.cpu().numpy())
    
    epoch_loss = running_loss / len(test_loader)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc, all_preds, all_labels


# Set the device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model
model = CNNTransformerHybrid(
    input_dim=187,  # Number of features per sample (input dimension)
    num_classes=10,  # Number of output classes
    num_heads=8,  # Multi-head attention heads
    num_layers=6  # Number of Transformer layers
).to(device)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()  # Suitable for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Optimizer with a learning rate of 1e-4

# Number of epochs to train
epochs = 10

# Training and testing loops
for epoch in range(epochs):
    # Train the model
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    
    # Test the model
    test_loss, test_acc, preds, labels = test(model, test_loader, criterion, device)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

# Optionally: Save the trained model
# torch.save(model.state_dict(), "cnn_transformer_model.pth")


Epoch 1/10
Train Loss: 0.3072, Train Accuracy: 0.9156
Test Loss: 0.1972, Test Accuracy: 0.9475
Epoch 2/10
Train Loss: 0.1844, Train Accuracy: 0.9495
Test Loss: 0.1562, Test Accuracy: 0.9572
Epoch 3/10
Train Loss: 0.1511, Train Accuracy: 0.9573
Test Loss: 0.1246, Test Accuracy: 0.9649
Epoch 4/10
Train Loss: 0.1262, Train Accuracy: 0.9651
Test Loss: 0.1078, Test Accuracy: 0.9702
Epoch 5/10
Train Loss: 0.1115, Train Accuracy: 0.9685
Test Loss: 0.1158, Test Accuracy: 0.9678
Epoch 6/10
Train Loss: 0.1007, Train Accuracy: 0.9714
Test Loss: 0.1364, Test Accuracy: 0.9653
Epoch 7/10
Train Loss: 0.0933, Train Accuracy: 0.9740
Test Loss: 0.0961, Test Accuracy: 0.9739
Epoch 8/10
Train Loss: 0.0887, Train Accuracy: 0.9748
Test Loss: 0.0843, Test Accuracy: 0.9767
Epoch 9/10
Train Loss: 0.0837, Train Accuracy: 0.9761
Test Loss: 0.0870, Test Accuracy: 0.9758
Epoch 10/10
Train Loss: 0.0806, Train Accuracy: 0.9769
Test Loss: 0.0809, Test Accuracy: 0.9771


In [82]:
model_path = "./model/cnn_transformer_model.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Model saved to ./model/cnn_transformer_model.pth


Testing the model

In [85]:
test_df = dataframes['mitbih_test']
print(test_df.shape)

(21892, 188)


In [97]:
labels = {
    0.0: "N",
    1.0: "S",
    2.0: "V",
    3.0: "F",
    4.0: "Q"
}

train_df.iloc[:, -1] = train_df.iloc[:, -1].replace(labels)

x_data = train_df.iloc[:,:187]
y_label = train_df.iloc[:,-1]

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_label.values.ravel())


# Prepare data for model
X = x_data
X_test = np.expand_dims(X, axis=1)    # Add a new dimension to match model input
X_test_tensor = torch.tensor(X_test).float()    # Convert to float tensor
y_test_tensor = torch.tensor(y).long()          # Convert to long tensor for labels

# Create DataLoader
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [98]:
model.eval()
correct_predictions = 0
total_predictions = 0

# Evaluate the model
def test(model, test_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():  # No need to compute gradients during testing
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            # Forward pass
            output = model(data)
            
            # Calculate the loss
            loss = criterion(output, target)
            running_loss += loss.item()
            
            # Get the predicted class
            _, predicted = torch.max(output, 1)
            
            total += target.size(0)
            correct += (predicted == target).sum().item()
            
            # Save predictions and labels for additional evaluation if needed
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(target.cpu().numpy())
    
    epoch_loss = running_loss / len(test_loader)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc, all_preds, all_labels


In [99]:
criterion = torch.nn.CrossEntropyLoss()  # For multi-class classification

epoch_loss, epoch_acc, all_preds, all_labels = test(model, test_loader, criterion, device)
print(f"Test Loss: {epoch_loss:.4f}, Test Accuracy: {epoch_acc:.4f}")

Test Loss: 0.0685, Test Accuracy: 0.9801
